In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import pandas as pd
import numpy as np

import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from cataclop.ml import preprocessing
from cataclop.ml import exploration

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)

from cataclop.ml.pipeline import factories

In [2]:
program = factories.Program.factory('default', version='1.1')

In [5]:
program.train(dataset_params = {
    'from': '2019-01-01',
    'to': '2019-12-31'
}, model_params = {
    'seed': 123456,
    'kfolds': 2,
    'nan_flag': -10000,
    'n_targets': 1,
}, dataset_reload=True)

12750 races


preparing model data
training nn_1


Epoch 1/10
30948/30948 [==============================] - 1s 27us/step - loss: 0.0972: 0s - 
Epoch 2/10
30948/30948 [==============================] - 1s 22us/step - loss: 0.0933
Epoch 3/10
30948/30948 [==============================] - 1s 24us/step - loss: 0.0922
Epoch 4/10
30948/30948 [==============================] - 1s 24us/step - loss: 0.0916
Epoch 5/10
30948/30948 [==============================] - 1s 22us/step - loss: 0.0913
Epoch 6/10
30948/30948 [==============================] - 1s 22us/step - loss: 0.0912
Epoch 7/10
30948/30948 [==============================] - 1s 22us/step - loss: 0.0911
Epoch 8/10
30948/30948 [==============================] - 1s 26us/step - loss: 0.0910
Epoch 9/10
30948/30948 [==============================] - 1s 22us/step - loss: 0.0910
Epoch 10/10
30949/30949 [==============================] - 0s 4us/step


mea: 0.259681295388739
training nn_51


Epoch 1/10
30948/30948 [==============================] - 1s 19us/step - loss: 0.0982
Epoch 2/10
30948/30948 [==============================] - 1s 23us/step - loss: 0.0924
Epoch 3/10
30948/30948 [==============================] - 1s 21us/step - loss: 0.0919
Epoch 4/10
30948/30948 [==============================] - 1s 21us/step - loss: 0.0916
Epoch 5/10
30948/30948 [==============================] - 1s 21us/step - loss: 0.0921
Epoch 6/10
30948/30948 [==============================] - 1s 23us/step - loss: 0.0917: 0s - loss: 0
Epoch 7/10
30948/30948 [==============================] - 1s 21us/step - loss: 0.0914
Epoch 8/10
30948/30948 [==============================] - 1s 21us/step - loss: 0.0913: 0s - loss:
Epoch 9/10
30948/30948 [==============================] - 1s 21us/step - loss: 0.0912
Epoch 10/10
30949/30949 [==============================] - 0s 3us/step


...

mea: 0.25245539662563693
training stacked_nn_7


Epoch 1/10
30949/30949 [==============================] - 1s 17us/step - loss: 0.0898
Epoch 2/10
30949/30949 [==============================] - 1s 22us/step - loss: 0.0880
Epoch 3/10
30949/30949 [==============================] - 1s 20us/step - loss: 0.0877
Epoch 4/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0876
Epoch 5/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0875
Epoch 6/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0874
Epoch 7/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0874
Epoch 8/10
30949/30949 [==============================] - 1s 20us/step - loss: 0.0874
Epoch 9/10
30949/30949 [==============================] - 1s 18us/step - loss: 0.0873
Epoch 10/10
30948/30948 [==============================] - 0s 3us/step


mea: 0.25398666093075833
training stacked_nn_9


Epoch 1/10
30949/30949 [==============================] - 1s 17us/step - loss: 0.0895
Epoch 2/10
30949/30949 [==============================] - 1s 17us/step - loss: 0.0881
Epoch 3/10
30949/30949 [==============================] - 1s 17us/step - loss: 0.0878
Epoch 4/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0877
Epoch 5/10
30949/30949 [==============================] - 1s 18us/step - loss: 0.0875
Epoch 6/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0874
Epoch 7/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0874
Epoch 8/10
30949/30949 [==============================] - 1s 22us/step - loss: 0.0874
Epoch 9/10
30949/30949 [==============================] - 1s 19us/step - loss: 0.0873
Epoch 10/10
30948/30948 [==============================] - 0s 3us/step


mea: 0.2523185959410488


In [6]:
program.df.describe()

,id,race_id,age,num,race_count,victory_count,placed_count,placed_2_count,placed_3_count,earnings,victory_earnings,placed_earnings,year_earnings,prev_year_earnings,post_position,position,handicap_weight,handicap_distance,trueskill_mu,trueskill_sigma,time,hist_1_days,hist_2_days,hist_3_days,winner_dividend,placed_dividend,final_odds,final_odds_offline,final_odds_unibet,final_odds_ref,final_odds_ref_offline,final_odds_ref_unibet,trainer_winning_rate,trainer_avg_winning_dividend,horse_id,trainer_id,jockey_id,herder_id,owner_id,num_race,num_bis,prize,distance,declared_player_count,session_id,num_session,hippodrome_id,hist_1_pos,hist_2_pos,hist_3_pos,hist_4_pos,hist_5_pos,hist_6_pos,speed,win,final_odds_ref_inv,race_count_mean,race_count_std,race_count_amin,race_count_amax,hist_1_pos_mean,hist_1_pos_std,hist_1_pos_amin,hist_1_pos_amax,hist_2_pos_mean,hist_2_pos_std,hist_2_pos_amin,hist_2_pos_amax,hist_3_pos_mean,hist_3_pos_std,hist_3_pos_amin,hist_3_pos_amax,post_position_mean,post_position_std,post_position_amin,post_position_amax,victory_count_mean,victory_count_std,victory_count_amin,victory_count_amax,placed_2_count_mean,placed_2_count_std,placed_2_count_amin,placed_2_count_amax,placed_3_count_mean,placed_3_count_std,placed_3_count_amin,placed_3_count_amax,victory_earnings_mean,victory_earnings_std,victory_earnings_amin,victory_earnings_amax,placed_earnings_mean,placed_earnings_std,placed_earnings_amin,placed_earnings_amax,prev_year_earnings_mean,prev_year_earnings_std,prev_year_earnings_amin,prev_year_earnings_amax,...,hist_2_days_amax,hist_3_days_mean,hist_3_days_std,hist_3_days_amin,hist_3_days_amax,trainer_winning_rate_mean,trainer_winning_rate_std,trainer_winning_rate_amin,trainer_winning_rate_amax,trainer_avg_winning_dividend_mean,trainer_avg_winning_dividend_std,trainer_avg_winning_dividend_amin,trainer_avg_winning_dividend_amax,race_count_r,hist_1_pos_r,hist_2_pos_r,hist_3_pos_r,post_position_r,victory_count_r,placed_2_count_r,placed_3_count_r,victory_earnings_r,placed_earnings_r,prev_year_earnings_r,handicap_distance_r,handicap_weight_r,final_odds_ref_r,final_odds_ref_offline_r,trueskill_mu_r,trueskill_sigma_r,hist_1_days_r,hist_2_days_r,hist_3_days_r,trainer_winning_rate_r,trainer_avg_winning_dividend_r,race_winner_dividend,odds_0,odds_1,odds_2,odds_3,odds_4,odds_5,odds_6,odds_7,odds_8,odds_9,odds_10,odds_11,odds_12,odds_13,odds_14,odds_15,odds_16,odds_17,odds_18,odds_19,target_returns,target,target_stacked,pred_nn_1_1,pred_nn_51_1,pred_nn_101_1,pred_nn_151_1,pred_xgb_10_1,pred_xgb_30_1,pred_xgb_50_1,pred_xgb_100_1,pred_knn_5_1,pred_nn_1_std,pred_nn_1_min,pred_nn_1_max,pred_nn_51_std,pred_nn_51_min,pred_nn_51_max,pred_nn_101_std,pred_nn_101_min,pred_nn_101_max,pred_nn_151_std,pred_nn_151_min,pred_nn_151_max,pred_xgb_10_std,pred_xgb_10_min,pred_xgb_10_max,pred_xgb_30_std,pred_xgb_30_min,pred_xgb_30_max,pred_xgb_50_std,pred_xgb_50_min,pred_xgb_50_max,pred_xgb_100_std,pred_xgb_100_min,pred_xgb_100_max,pred_knn_5_std,pred_knn_5_min,pred_knn_5_max,pred_stacked_stacked_nn_1_1,pred_stacked_stacked_nn_3_1,pred_stacked_stacked_nn_5_1,pred_stacked_stacked_nn_7_1,pred_stacked_stacked_nn_9_1
count,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,6.189700e+04,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,51458.000000,61897.000000,61897.000000,61897.000000,61897.000000,30613.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61273.000000,53368.000000,52659.000000,61897.000000,61897.000000,52544.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,52888.000000,61897.000000,61897.000000,11127.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,61897.000000,6189

In [7]:
program.model.features

['declared_player_count',
 'handicap_distance',
 'handicap_distance_amax',
 'handicap_distance_amin',
 'handicap_distance_mean',
 'handicap_distance_r',
 'handicap_distance_std',
 'handicap_weight',
 'handicap_weight_amax',
 'handicap_weight_amin',
 'handicap_weight_mean',
 'handicap_weight_r',
 'handicap_weight_std',
 'hist_1_days',
 'hist_1_days_amax',
 'hist_1_days_amin',
 'hist_1_days_mean',
 'hist_1_days_r',
 'hist_1_days_std',
 'hist_1_pos',
 'hist_1_pos_amax',
 'hist_1_pos_amin',
 'hist_1_pos_mean',
 'hist_1_pos_r',
 'hist_1_pos_std',
 'hist_2_days',
 'hist_2_days_amax',
 'hist_2_days_amin',
 'hist_2_days_mean',
 'hist_2_days_r',
 'hist_2_days_std',
 'hist_2_pos',
 'hist_2_pos_amax',
 'hist_2_pos_amin',
 'hist_2_pos_mean',
 'hist_2_pos_r',
 'hist_2_pos_std',
 'hist_3_days',
 'hist_3_days_amax',
 'hist_3_days_amin',
 'hist_3_days_mean',
 'hist_3_days_r',
 'hist_3_days_std',
 'hist_3_pos',
 'hist_3_pos_amax',
 'hist_3_pos_amin',
 'hist_3_pos_mean',
 'hist_3_pos_r',
 'hist_3_pos_st

In [8]:
program.df.shape, len(program.model.features)

((61897, 255), 135)

In [9]:
[model["name"] for model in program.model.models] + [model["name"] for model in program.model.stacked_models]

['nn_1',
 'nn_51',
 'nn_101',
 'nn_151',
 'xgb_10',
 'xgb_30',
 'xgb_50',
 'xgb_100',
 'knn_5',
 'stacked_nn_1',
 'stacked_nn_3',
 'stacked_nn_5',
 'stacked_nn_7',
 'stacked_nn_9']

In [10]:
print(
    '{} races from {} to {}'.format(
        program.df.race_id.nunique(), 
        program.df.start_at.min(), 
        program.df.start_at.max()
    )
)

4754 races from 2019-01-07 12:25:00 to 2019-12-30 20:33:00


In [11]:
for model in program.model.models:
    clf = model['estimators'][-1]['pipeline'].steps[-1][1]
    if hasattr(clf, 'classes_'):
        clf = model['estimators'][-1]['pipeline'].steps[-1][1]
        print(clf.classes_)

In [12]:
pd.set_option('precision',7)
    
for model in program.model.models:
    clf = model['estimators'][-1]['pipeline'].steps[-1][1]
    if hasattr(clf, 'feature_importances_'):
        fi = pd.Series(clf.feature_importances_, index=program.model.features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
        print(model['name'])
        print(fi.sort_values(ascending=False)[0:100])
        print("---\n")
    if hasattr(clf, 'coef_'):
        fi = pd.Series(clf.coef_, index=program.model.features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
        print(model['name'])
        print(fi.sort_values(ascending=False)[0:100])
        print("---\n")

if hasattr(program.model, "stacked_models"):
    for model in program.model.stacked_models:
        clf = model['estimators'][-1]['pipeline'].steps[-1][1]
        if hasattr(clf, 'feature_importances_'):
            fi = pd.Series(clf.feature_importances_, index=program.model.stacked_features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
            print(model['name'])
            print(fi.sort_values(ascending=False)[0:100])
            print("---\n")
        if hasattr(clf, 'coef_'):
            fi = pd.Series(clf.coef_, index=program.model.stacked_features+preprocessing.get_dummy_features(model['estimators'][-1]['dummies']))
            print(model['name'])
            print(fi.sort_values(ascending=False)[0:100])
            print("---\n")
        
pd.set_option('precision',7)



xgb_10
hist_1_pos_r                   0.3350336
hist_2_pos_r                   0.0837714
hist_3_pos_r                   0.0703592
hist_4_pos                     0.0700453
prev_year_earnings             0.0651437
prev_year_earnings_r           0.0614955
race_count_r                   0.0586898
victory_earnings_r             0.0466049
race_count                     0.0418845
hist_1_pos                     0.0392210
trueskill_mu_r                 0.0383427
hist_5_pos                     0.0378077
victory_earnings               0.0221650
hist_6_pos                     0.0180874
hist_2_pos_std                 0.0113483
hist_3_pos                     0.0000000
odds_0                         0.0000000
hist_3_pos_mean                0.0000000
odds_4                         0.0000000
odds_1                         0.0000000
odds_2                         0.0000000
hist_3_pos_amax                0.0000000
odds_3                         0.0000000
hist_3_pos_amin                0.0000000
hist_3_po

In [13]:
df = program.df

for model in program.model.models:
    print(model['name'], df['pred_{}_1'.format(model['name'])].mean())

nn_1 0.5582643236240121
nn_51 0.5560011830025711
nn_101 0.5599908332322041
nn_151 0.5576331542274162
xgb_10 0.5370656806469264
xgb_30 0.5545959675071493
xgb_50 0.5567778975360352
xgb_100 0.5571842364370915
knn_5 0.5546809238960925


In [14]:
from sklearn import preprocessing

df['pred_sum_1'] = 1
for model in program.model.models:
    m = model['name']
    #s = preprocessing.MinMaxScaler()
    #scaled = s.fit_transform(df[['pred_{}_1'.format(m)]].values)
    #df['pred_{}_s_1'.format(m)] = scaled
    #df['pred_sum_1'] = df['pred_sum_1'] * df['pred_{}_s_1'.format(m)]
    
df['pred_sum_1'] = df[['pred_{}_1'.format(model['name']) for model in program.model.models]].sum(axis=1)
df['pred_sum_1'] /= len(program.model.models)
df['pred_stacked_sum_1'] = df[['pred_stacked_{}_1'.format(model['name']) for model in program.model.stacked_models]].sum(axis=1)
df['pred_stacked_sum_1'] /= len(program.model.stacked_models)
df['pred_odds_1'] = df['final_odds_ref']
df['pred_rnd_1'] = np.random.rand(df.shape[0])
df['pred_trueskill_mu_1'] = df['trueskill_mu']

In [15]:
cols = [
            'position', 
            'declared_player_count', 
            'sub_category', 
            'num', 
            'final_odds', 
            'final_odds_ref', 
            'final_odds_ref_unibet', 
            'target',
            'pred_sum_1',
            'pred_stacked_sum_1',
            'race_winner_dividend',
            'winner_dividend',
            'placed_dividend',
            'trueskill_mu',
            'hist_1_pos'
        ] + [
            'pred_{}_1'.format(model['name']) for model in program.model.models
        ] + [
            'pred_stacked_{}_1'.format(model['name']) for model in program.model.stacked_models
        ] 
exploration.random_race(df, 
                        cols=cols
                       ).sort_values(by='position', ascending=True)

position  declared_player_count     sub_category  num  \
race_id                                                                
71661   27645       1.0                     15  HANDICAP_DIVISE    4   
        27649       2.0                     15  HANDICAP_DIVISE    8   
        27652       3.0                     15  HANDICAP_DIVISE   11   
        27653       4.0                     15  HANDICAP_DIVISE   12   
        27644       5.0                     15  HANDICAP_DIVISE    3   
        27656       6.0                     15  HANDICAP_DIVISE   15   
        27655       7.0                     15  HANDICAP_DIVISE   14   
        27646       8.0                     15  HANDICAP_DIVISE    5   
        27643       9.0                     15  HANDICAP_DIVISE    2   
        27651      10.0                     15  HANDICAP_DIVISE   10   
        27647      11.0                     15  HANDICAP_DIVISE    6   
        27648      12.0                     15  HANDICAP_DIVISE    7   
        27650      13.0                     15  HANDICAP_DIVISE    9   
        27654      14.0                     15  HANDICAP_DIVISE   13   
        27642      15.0                     15  HANDICAP_DIVISE    1   

               final_odds  final_odds_ref  final_odds_ref_unibet     target  \
race_id                                                                       
71661   27645         5.0             4.9                    4.7  0.0666667   
        27649        11.0            14.0                   10.2  0.1333333   
        27652        11.0            14.0                    8.0  0.2000000   
        27653        23.0            34.0                   22.5  0.2666667   
        27644        19.0            15.0                   19.2  0.3333333   
        27656        26.0            19.0                   35.3  0.4000000   
        27655         7.0            10.0                    5.4  0.4666667   
        27646         6.5             8.7                    6.7  0.5333333   
        27643        11.0            13.0                   15.2  0.6000000   
        27651        16.0             8.6                   20.3  0.6666667   
        27647        16.0            11.0                   12.4  0.7333333   
        27648        22.0            17.0                   33.4  0.8000000   
        27650        14.0            13.0                   21.2  0.8666667   
        27654        37.0            22.0                   43.6  0.9333333   
        27642        56.0            56.0                   98.6  1.0000000   

               pred_sum_1  pred_stacked_sum_1  race_winner_dividend  \
race_id                                                               
71661   27645   0.4443989           0.3814573                 340.0   
        27649   0.5260454           0.5109696                 340.0   
        27652   0.5979175           0.6203244                 340.0   
        27653   0.4539852           0.4450741                 340.0   
        27644   0.5715168           0.5530646                 340.0   
        27656   0.6020795           0.6388102                 340.0   
        27655   0.4588670           0.4604074                 340.0   
        27646   0.5144313           0.5222890                 340.0   
        27643   0.5232965           0.5109978                 340.0   
        27651   0.4949845           0.4750927                 340.0   
        27647   0.5016317           0.4682142                 340.0   
        27648   0.6344348           0.6519622                 340.0   
        27650   0.5204036           0.5099729                 340.0   
        27654   0.6355349           0.6855708                 340.0   
        27642   0.5518105           0.5019595                 340.0   

               winner_dividend  placed_dividend  trueskill_mu  hist_1_pos  \
race_id                                                                     
71661   27645            340.0            170.0    25.0000000         2.0   
       

In [ ]:
# more exploration of a single race features
#(df.reset_index(drop=True).set_index(['race_id', df.index]).loc[65509][['position'] + program.model.features]).sort_values(by='position', ascending=True)

In [ ]:
models = program.model.models + [{"name": "sum"}, {"name": "stacked_sum"}, {"name": "rnd"}, {"name": "odds"}, {"name": "trueskill_mu"}]
#dd = df.groupby('race_id').filter(lambda r: r['pred_sum'].std()!=0)
models = models + [{"name": "stacked_{}".format(model["name"])} for model in program.model.stacked_models]
dd = df
for model in models:
    m = model['name']
    dd['bet_{}'.format(m)] = 0.
    dd['profit_{}'.format(m)] = 0.
    dd['n_{}'.format(m)] = 0.
    dd['n_odds_{}'.format(m)] = 0.
    dd['pred_{}_std'.format(m)] = 0.
    

In [ ]:
dd['pred_rnd_1'] = np.random.rand(dd.shape[0])

In [ ]:
[model["name"] for model in models]

In [ ]:

def fast_bet(r):
    for model in models:
        m = model['name']
        p = 'pred_{}_1'.format(m)
        #print(model['name'], df['pred_{}_1'.format(model['name'])].mean())
        s = r.sort_values(by=p, ascending=False)
        o = s.index.sort_values(ascending=True, return_indexer=True)
        s2 = r.sort_values(by='final_odds_ref')
        o2 = s2.index.sort_values(ascending=True, return_indexer=True)
        
        
        idx = (r[p] == r[p].max())
    #idx = (r['pred_knn_5_1'] > 0) & (r['final_odds_ref'] > 5)
    #idx = (r['pred_knn_5_1'] > 0.) & (r['final_odds_ref'] > 5) & (r['final_odds_ref'] < 30)
        #if r[p].std() == 0:
        #    r['bet'] = 0
        #    return r
        r['bet_{}'.format(m)] = np.clip(r[p], a_min=0., a_max=1.) #((idx).astype('float'))
        
        
        r['n_{}'.format(m)] = o[1]
        r['n_odds_{}'.format(m)] = o2[1]
        r['pred_{}_std'.format(m)] = r[p].std()
    return r
    

dd = dd.groupby('race_id').apply(fast_bet)

for model in models:
    dd['profit_{}'.format(model['name'])] = dd['bet_{}'.format(model['name'])] * 1.0 * (dd['target_returns']-1.0)

cols = ['profit_{}'.format(model['name']) for model in models]

    

In [ ]:
for model in models:
    m = model['name']
    #dd['profit_{}'.format(m)] = np.clip(dd['pred_{}_1'.format(m)], a_min=0., a_max=10.) * 1.0 * (dd['target_returns']-1.0)
    #dd['profit_{}'.format(m)] = 1.0 * (dd['target_returns']-1.0)
    dd['bet_{}'.format(m)] = 1#dd['pred_{}_1'.format(m)]#np.ceil( 1 * 1./(dd['n_{}'.format(m)]+1.) * ( np.clip(dd['pred_{}_1'.format(m)], a_min=0., a_max=10.) * np.log(dd['n_odds_{}'.format(m)]+1.) ) )
    #dd['bet_{}'.format(m)] = np.clip((dd['pred_{}_1'.format(m)]/10.), a_min=0., a_max=100.)
    dd['profit_{}'.format(m)] =  dd['bet_{}'.format(m)] * (dd['winner_dividend'].fillna(0.)/100.-1.0)
    

In [ ]:
cols = [
            'position', 
            'declared_player_count', 
            'sub_category', 
            'num', 
            'final_odds', 
            'final_odds_ref', 
            'final_odds_ref_unibet', 
            'target',
            'race_winner_dividend',
            'trueskill_mu',
        ] + [
            'n_{}'.format(model['name']) for model in models
        ] + [
            'pred_{}_1'.format(model['name']) for model in models
        ] + [
            'bet_{}'.format(model['name']) for model in models
        ] + [
            'profit_{}'.format(model['name']) for model in models
        ]
exploration.random_race(dd, 
                        cols=cols
                       ).sort_values(by='position', ascending=True)

In [ ]:
#s = dd.groupby('race_id')['final_odds_offline'].std()
#s = s.to_frame()
#s.columns = ['final_odds_offline_std']

In [ ]:
#dd = dd.join(s, how='left', on='race_id')

In [ ]:
#len(dd[(dd['start_at']>'2019-06-01') & (dd['start_at']<'2019-08-01')])

In [ ]:
r = pd.DataFrame(columns=['r', 'bets_mean', 'bets_max', 'bets_min', 'profit_mean', 'profit_max', 'profit_min', 'stash_min', 'stash_max', 'count'], index=[model['name'] for model in models])
for model in models:
#for model in [{"name": "stacked_sum"}, {"name": "sum"}]:
    m = model['name']
    
    #for c in dd['sub_category'].value_counts().sort_values(ascending=False).index[0:7]:
    #print(c)

    ddd = dd[(dd['sub_category'].isin(['HANDICAP'])) ]#dd[(dd['sub_category']==c)]# & (dd['sub_category'].isin(['HANDICAjP_DIVISE', 'HANjDICAP', 'AUTOSTART'])) ]
    #ddd = ddd.groupby('race_id').filter(lambda r: r['pred_{}_1'.format(m)].std() > 0.01)
    #g = ddd[(ddd['n_odds_{}'.format(m)]>=0) & (ddd['pred_{}_1'.format(m)] >= np.quantile(ddd['pred_{}_1'.format(m)], 0.7)) & (ddd['final_odds_ref_offline'].notnull()) & (ddd['final_odds_ref_offline'] > ddd['final_odds_offline']) & (ddd['final_odds_ref'] > 20) & (ddd['final_odds_ref']<50)][['start_at', 'race_id', 'position', 'final_odds', 'final_odds_ref', 'final_odds_ref_unibet', 'final_odds_ref_offline_std', 'final_odds_offline_std', 'profit_{}'.format(m), 'bet_{}'.format(m), 'pred_{}_1'.format(m)]].copy()
    g = ddd[ (ddd['pred_{}_std'.format(m)] > 0. ) & (ddd['n_{}'.format(m)] == 0) & (ddd['final_odds_ref_offline'] > ddd['final_odds_offline']) & (ddd['final_odds_ref_offline'].notnull())  ][['start_at', 'race_id', 'position', 'final_odds', 'final_odds_ref', 'profit_{}'.format(m), 'bet_{}'.format(m), 'pred_{}_1'.format(m)]].copy()
    g['stash_{}'.format(m)] = g['profit_{}'.format(m)].cumsum()
    g['bets_{}'.format(m)] = g['bet_{}'.format(m)].cumsum()
    g['stash_{}'.format(m)].plot(figsize=(16, 4), title=m)
    g['bets_{}'.format(m)].plot()
    plt.grid(True)
    plt.show()

    #print(m, np.quantile(ddd['pred_{}_1'.format(m)], 0.7), ddd['start_at'].min(), ddd['start_at'].max())

    r.loc[m]['r'] = g['profit_{}'.format(m)].sum() / g['bet_{}'.format(m)].sum()
    r.loc[m]['bets_mean'] = g['bet_{}'.format(m)].mean()
    r.loc[m]['bets_min'] = g['bet_{}'.format(m)].min()
    r.loc[m]['bets_max'] = g['bet_{}'.format(m)].max()
    r.loc[m]['profit_mean'] = g['profit_{}'.format(m)].mean()
    r.loc[m]['profit_min'] = g['profit_{}'.format(m)].min()
    r.loc[m]['profit_max'] = g['profit_{}'.format(m)].max()
    r.loc[m]['stash_min'] = g['stash_{}'.format(m)].min()
    r.loc[m]['stash_max'] = g['stash_{}'.format(m)].max()
    r.loc[m]['count'] = g['stash_{}'.format(m)].count()

print(r)
    
    

In [ ]:
np.quantile(ddd['pred_{}_1'.format(m)], 0.7)

In [ ]:
def bet(df):
    models = [{"name":'mlp_30'}]
    
    def fast_bet(r):
        for model in models:
            p = 'pred_{}_1'.format(model['name'])
            #print(model['name'], df['pred_{}_1'.format(model['name'])].mean())
            s = r.sort_values(by=p)
            o = s.index.sort_values(ascending=True, return_indexer=True)
            s2 = r.sort_values(by='final_odds_ref')
            o2 = s2.index.sort_values(ascending=True, return_indexer=True)


            idx = (r[p] == r[p].max())
        #idx = (r['pred_knn_5_1'] > 0) & (r['final_odds_ref'] > 5)
        #idx = (r['pred_knn_5_1'] > 0.) & (r['final_odds_ref'] > 5) & (r['final_odds_ref'] < 30)
            #if r[p].std() == 0:
            #    r['bet'] = 0
            #    return r
            r['bet_{}'.format(model['name'])] = np.clip(r[p], a_min=0., a_max=1.) #((idx).astype('float'))


            r['n_{}'.format(model['name'])] = o[1]
            r['n_odds_{}'.format(model['name'])] = o2[1]
        return r
    
    df = df[(df['country']=='FRA') & (df['sub_category'].isin(['HANDICAP', 'HANDICAP_DIVISE']))].copy()
    df = df.groupby('race_id').apply(fast_bet)

    for model in models:
        m = model['name']
        #dd['profit_{}'.format(m)] = np.clip(dd['pred_{}_1'.format(m)], a_min=0., a_max=10.) * 1.0 * (dd['target_returns']-1.0)
        #dd['profit_{}'.format(m)] = 1.0 * (dd['target_returns']-1.0)
        df['bet_{}'.format(m)] = np.ceil(0.1 * np.clip((df['pred_{}_1'.format(m)]/10.), a_min=0., a_max=10.) * np.log(df['n_odds_{}'.format(m)]+1.) )
        df['profit_{}'.format(m)] = df['bet_{}'.format(m)] * 1.0 * (df['target_returns']-1.0)

    df['bet'] = df[['bet_{}'.format(model['name']) for model in models]].sum(axis=1)
    df['profit'] = df[['profit_{}'.format(model['name']) for model in models]].sum(axis=1)
    df['target'] = 'mlp_30_1'
    
    bets = df[(df['pred_mlp_30_1'] >= 13.6) & (df['final_odds_ref_offline']>df['final_odds_offline']) & (df['final_odds_ref'] > 20) & (df['final_odds_ref']<50)][['race_id', 'start_at', 'bet', 'category', 'sub_category', 'country', 'profit', 'num', 'race_id', 'position', 'final_odds_ref', 'final_odds', 'profit_{}'.format(m), 'bet_{}'.format(m), 'pred_{}_1'.format(m)]].copy()
    bets['date'] = pd.to_datetime(bets['start_at'])
    
    #bets = bets.set_index(bets['date'])
    #bets = bets.sort_index()
    
    bets['bets'] = bets['bet'].cumsum()
    bets['stash'] = bets['profit'].cumsum()

    return bets

In [ ]:
#bets = bet(df, program.model.features, program.model.categorical_features, 
#    N=1, max_odds=30, break_on_bet=True, break_on_odds=False, 
#    targets=['pred_rnd_1', 'final_odds_ref', 'pred_sum_1'] + ['pred_{}_{}'.format(model['name'], i+1) for i in range(program.model.params['n_targets']) for model in program.model.models]# + ['pred_stacked_{}_1'.format(model['name']) for model in program.model.stacked_models] 
#   )   

In [ ]:
bets = bet(df)

In [ ]:
#bets = bets.set_index(bets['date'])
#bets = bets.sort_index()

In [ ]:
list(bets.columns)

In [ ]:
def graph_bb(bb, f):
    results = []

    for s in bb[f].value_counts().index:
        results = []
        x = bb[ (bb[f] == s)  ].copy()
        if len(x) == 0:
            continue
        #print("---\n{}\t{:+.2f}\t{:+.2f}\t{:+.2f}\n---".format(s, x['profit'].sum(), x['bet'].sum(), len(x)))

        fig, axs = plt.subplots(1,1)

        for t in x['target'].value_counts().index:
            bbb = x[(x['target'] == t)].copy()
            #bbb = bbb[ (bbb['pred']>bbb['pred'].quantile(0.85)) ].copy()


            results.append({
                'idx_1': s,
                'idx_2': t,
                'profit': bbb.profit.sum(),
                'profit_place': bbb.profit_placed.sum(),
                'bet': bbb.bet.sum(),
                'bets': bbb.bet.count(),
                'avg': bbb.profit.sum() / bbb.bet.sum()
            })

            '''
            bbb['stash'] = bbb['profit'].cumsum()
            bbb['cbet'] = bbb['bet'].cumsum()

            fig, axs = plt.subplots(1,1)
            bbb['stash'].plot(figsize=(6, 2))
            bbb['cbet'].plot(figsize=(6, 2))
            plt.show()
            '''

            #print("{:10s}: \t {:+.2f} \t {:+.0f} \t {:+.2f}".format(t, bbb.profit.sum(), bbb.profit.count(), bbb.profit.mean()) )

            bbb['stash'] = bbb['profit'].cumsum()
            bbb['cbet'] = bbb['bet'].cumsum()

            bbb['stash'].plot(figsize=(10, 2))
            #bbb['cbet'].plot(figsize=(10, 2))

        results = pd.DataFrame(results)
        results = results.set_index(['idx_1', 'idx_2'])
        print(results.sort_values(by='avg', ascending=False))


        plt.show()
        

In [ ]:
#countries = list(bets['country'].value_counts()[0:10].index)
#sub_categories = list(bets['sub_category'].value_counts()[0:10].index)

#%store countries
#%store sub_categories

In [ ]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args

target = 'pred_knn_1_1'

c_min = bets[(bets['target'] == target)]['pred'].min()
c_mean = bets[(bets['target'] == target)]['pred'].mean()
c_max = bets[(bets['target'] == target)]['pred'].max()

space = [
          Real(1, 10, name='min_odds'),
          Real(10, 60, name='max_odds'),
          Real(c_min, c_mean, name='min_pred'),
          Real(c_mean, c_max, name='max_pred'),
          Integer(5, 25, name='max_players')] + [
          Integer(0,1, name='country_{}'.format(country)) for country in countries] + [
          Integer(0,1, name='sub_category_{}'.format(sub_category)) for sub_category in sub_categories] + [
          Integer(0,1, name='nb_{}'.format(n)) for n in range(0,6)]
         
def x_to_params(x):
    params = {}
    
    params['min_odds'] = x[0]
    params['max_odds'] = x[1]
    params['max_pred'] = x[3]
    params['min_pred'] = x[2]
    params['max_players'] = x[4]
    
    for i, country in enumerate(countries):
        params['country_{}'.format(country)] = x[5+i]
        
    for i, sc in enumerate(sub_categories):
        params['sub_category_{}'.format(sc)] = x[5+len(countries)+i]
        
    for n in range(0,6):
        params['nb_{}'.format(n)] = x[5+len(countries)+len(sub_categories)+n]

    return params

@use_named_args(space)
def f(**params):
    return ff(params)
    
def ff(params, train=True):
    
    print(params)
    
    b = bets[ (bets['pred'] != 0) & (bets['target']==target) & (bets['pred_std'] != 0)]
    
    b = b[ (b['odds_ref'] > params['min_odds']) & (b['odds_ref'] < params['max_odds']) & (b['declared_player_count'] > 1) & (b['declared_player_count'] <= params['max_players']) ]
    
    #b = b[(b['pred'] >= params['min_pred']) & (b['pred'] <= params['max_pred'])]
    
    for country in countries:
        if params['country_{}'.format(country)] == 0:
            b = b[ b['country'] != country ]
            
    for sub_category in sub_categories:
        if params['sub_category_{}'.format(sub_category)] == 0:
            b = b[ b['sub_category'] != sub_category ]
            
    for n in range(0,6):
        if params['nb_{}'.format(n)] == 0:
            b = b[ b['nb'] != n]
    
    p = b.profit.sum()
    pp = len(b[b['profit'] > 0])
    bet = b.bet.sum()
    
    print("{:+.2f} {:+.2f}\n".format(p, bet))
        
    if bet == 0:
        return 0
    
    if not train:
        return b
    
    return - (p)
    
    

In [ ]:
#c_mean, c_min, c_max

In [ ]:
#from skopt import dummy_minimize, gp_minimize, gbrt_minimize

#res = dummy_minimize(f, space, n_calls=500)

In [ ]:
#x_to_params(res.x), res.fun

In [ ]:
#params = x_to_params(res.x)
#%store params
#params

In [ ]:
#%store -r params
#%store -r countries
#%store -r sub_categories

In [ ]:
#bb = ff(params, False)

In [ ]:
#bb = bets[(bets['pred'] > 1.) & (bets['next_pred_1'] != bets['next_pred_2'])]
#bb = bets[(bets['pred_std'] < 0.1) & (bets['pred'] < 1.)]
#bb = bets[ (bets['odds_ref'] > 1) & (bets['odds_ref'] < 30) & (bets['pred'] > 1) & (bets['target'] == 'pred_xgb_100_1') ]
#graph_bb( bb[bb['target']=='pred_knn_1_1'], 'category')
#graph_bb( bets, 'category')

In [ ]:
bets = bets.iloc[:,~bets.columns.duplicated()]

In [ ]:
#bb = bets[(bets['sub_category']=='HANDICAP') & (bets['country'] != 'FRA')]
#bb = bb[(bb['pred_mlp_10_1']>118) & (bb['final_odds_ref']>=bb['final_odds'])].copy()
bb = bets
bb['stash'] = bb.profit.cumsum()
bb['stash'].plot(figsize=(12,4))

In [ ]:
# to save the program when the model looks good, call: program.lock('2020-03-03')

# a copy of the program, model and dataset class will be created

# after this, open the new program class in *ml/pipeline/programs* and update the `model_params` in the `run` function

# add any additional bet filter and bet strategy to the program bet function

# add the new program to the better command *pmu/management/commands/bet.py*: programs = ['2020-03-03']

In [16]:
program.lock('position_prediction')